In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## load dataset

In [4]:
data = pd.read_csv('dataset_sms_spam_v1.csv')
data.head()

,teks,label
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2


## teks preproccesng


## CASE FOLDING


In [17]:
import re

# membuat fungsi untuk casefolding
def casefolding(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[-+]?[0-9]+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text


In [18]:
#membandingkan before dan after case folding
raw_sample = data ['teks'].iloc[2]
case_folding = casefolding(raw_sample)

print('raw data\t : ', raw_sample)
print('Case Folding\t :', case_folding)

raw data	 :  2016-07-08 11:47:11.Plg Yth, sisa kuota Flash Anda 478KB. Download MyTelkomsel apps di http://tsel.me/tsel utk cek kuota&beli paket Flash atau hub *363#
Case Folding	 : plg yth sisa kuota flash anda  kb download mytelkomsel apps di  utk cek kuotabeli paket flash atau hub


## word normalization


In [19]:
key_norm = pd.read_csv('key_norm.csv')

def text_normalize(text):
    text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0]
    if (key_norm['singkat'] == word).any()
    else word for word in text.split()
    ])
    
    text = str.lower(text)
    return text

In [21]:
#membandingkan before dan after word normalization

raw_data = data['teks'].iloc[696]
word_normal = text_normalize(case_folding)

print('Raw Data\t :', raw_data)
print('Word Normalize\t :', word_normal)

Raw Data	 : Btw magicomnya yg sedang Gais, gaada yg gede
Word Normalize	 : pulang yang terhormat sisa kuota flash anda kb download mytelkomsel apps di untuk cek kuotabeli paket flash atau hubungi


## filtering (stopward Removal)

In [22]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_ind = stopwords.words('indonesian')


In [23]:
len(stopwords_ind)

758

In [24]:
#melihat daftar stopword dari nltk 

stopwords_ind

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [37]:
# membuat kata dalam stopword
more_stopword = ['tsel', 'gb', 'rb', 'btw']
stopwords_ind = stopwords_ind + more_stopword

def remove_stop_word(text):
    clean_words = []
    text = text.split()

    for word in text:
        if word not in stopwords_ind:
            clean_words.append(word)
    return " ".join(clean_words)


In [38]:
raw_sample = data['teks'].iloc[696]
case_folding = casefolding(raw_sample)
stopword_removal =  remove_stop_word(case_folding)

print('Raw Data \t\t :', raw_data)
print('Case Folding \t\t :', case_folding)
print('Stopword Removal \t\t: ', stopword_removal)

Raw Data 		 : Btw magicomnya yg sedang Gais, gaada yg gede
Case Folding 		 : btw magicomnya yg sedang gais gaada yg gede
Stopword Removal 		:  magicomnya yg gais gaada yg gede


# stemming


In [39]:
!pip -q install sastrawi

In [40]:
# merubah kata menjadi kata dasar
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

#membuat fungsi untuk stemming bahasa indonesia
def stemming(text):
    text = stemmer.stem(text)
    return text

In [41]:
raw_sample = data['teks'].iloc[696]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_word(case_folding)
text_stemming = stemming(stopword_removal)

print('Raw Data \t\t :', raw_sample)
print('Case Folding \t\t :', case_folding)
print('Stopword Removal \t\t :', stopword_removal)
print('Stemming \t\t :', text_stemming)

Raw Data 		 : Btw magicomnya yg sedang Gais, gaada yg gede
Case Folding 		 : btw magicomnya yg sedang gais gaada yg gede
Stopword Removal 		 : magicomnya yg gais gaada yg gede
Stemming 		 : magicomnya yg gais gaada yg gede


## text preproccesing pipeline


In [42]:
## membuat fungsi untuk menggabungkan seluruh langkah text preproccesing
def text_preprocessing_process(text):
    text = casefolding(text)
    text = text_normalize(text)
    text = remove_stop_word(text)
    text = stemming(text)
    return text

In [43]:
%%time
data['clean_teks']= data['teks'].apply(text_preprocessing_process)

CPU times: total: 3min 16s
Wall time: 13min 4s


In [45]:
data

,teks,label,clean_teks
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,2,promo beli paket flash my telkomsel app extra ...
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,2,rupiah ribu spesial pilih aktif promo sd novem...
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",2,pulang hormat sisa kuota flash kb download myt...
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",2,pulang hormat sisa kuota flash kb download myt...
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,2,rupiah ribu spesial pilih aktif buru skb
...,...,...,...
1138,"Yooo sama2, oke nanti aku umumin di grup kelas",0,yooo oke umumin grup kelas
1139,😁 sebelumnya ga ad nulis kerudung. Kirain warn...,0,nulis kerudung kirain warna jins
1140,Mba mau kirim 300 ya,0,mbak kirim ya
1141,nama1 beaok bwrangkat pagi...mau cas atay tra...,0,nama beaok bwrangkat pagimau cas atay tranfer


In [46]:
#simoan data yang sudah di preproccesing ke dalam file csv

data.to_csv('clean_data.csv')

## Feature Engineering

In [47]:
# pisahkan kolom feature dan target

x = data['clean_teks']
y = data['label']

In [48]:
x

0       promo beli paket flash my telkomsel app extra ...
1       rupiah ribu spesial pilih aktif promo sd novem...
2       pulang hormat sisa kuota flash kb download myt...
3       pulang hormat sisa kuota flash kb download myt...
4                rupiah ribu spesial pilih aktif buru skb
                              ...                        
1138                           yooo oke umumin grup kelas
1139                     nulis kerudung kirain warna jins
1140                                        mbak kirim ya
1141        nama beaok bwrangkat pagimau cas atay tranfer
1142                                       nomor bri nama
Name: clean_teks, Length: 1143, dtype: object

In [49]:
y

0       2
1       2
2       2
3       2
4       2
       ..
1138    0
1139    0
1140    0
1141    0
1142    0
Name: label, Length: 1143, dtype: int64

## feature extraction (TF-IDF dan N-Gram)

In [77]:
#save model
import pickle

#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

#unigram
vec_TF_IDF = TfidfVectorizer(ngram_range=(1,1))
vec_TF_IDF.fit(x)

x_tf_idf = vec_TF_IDF.transform(x)

pickle.dump(vec_TF_IDF.vocabulary_,open("features_tf-idf.sav", "wb"))

In [78]:
#menampilkan vocabulary dari tf-idf
vec_TF_IDF.vocabulary_

{'promo': 2222,
 'beli': 306,
 'paket': 2036,
 'flash': 842,
 'my': 1834,
 'telkomsel': 2783,
 'app': 149,
 'extra': 813,
 'kuota': 1505,
 'lte': 1607,
 'telpon': 2786,
 'buru': 462,
 'cek': 503,
 'tselmemytsel': 2916,
 'sk': 2600,
 'rupiah': 2412,
 'ribu': 2378,
 'spesial': 2658,
 'pilih': 2119,
 'aktif': 64,
 'sd': 2467,
 'november': 1938,
 'pulang': 2253,
 'hormat': 1077,
 'sisa': 2593,
 'kb': 1367,
 'download': 726,
 'mytelkomsel': 1836,
 'apps': 154,
 'kuotabeli': 1506,
 'hubung': 1097,
 'skb': 2601,
 'ekstra': 778,
 'pulsa': 2255,
 'internet': 1176,
 'bulan': 448,
 'sjk': 2599,
 'aug': 203,
 'september': 2525,
 'detail': 643,
 'iring': 1198,
 'tarif': 2749,
 'panjang': 2049,
 'hits': 1066,
 'armada': 167,
 'curi': 582,
 'hati': 1031,
 'tekan': 2777,
 'okcall': 1989,
 'informasi': 1147,
 'eks': 775,
 'loh': 1595,
 'internetan': 1177,
 'pakai': 2034,
 'volume': 3032,
 'ultima': 2970,
 'mbak': 1692,
 'harga': 1019,
 'tariflokasi': 2751,
 'tselmefl': 2914,
 'coboy': 550,
 'jr': 1284,

In [79]:
print('hasil vocabulary :', vec_TF_IDF.vocabulary_)

hasil vocabulary : {'promo': 2222, 'beli': 306, 'paket': 2036, 'flash': 842, 'my': 1834, 'telkomsel': 2783, 'app': 149, 'extra': 813, 'kuota': 1505, 'lte': 1607, 'telpon': 2786, 'buru': 462, 'cek': 503, 'tselmemytsel': 2916, 'sk': 2600, 'rupiah': 2412, 'ribu': 2378, 'spesial': 2658, 'pilih': 2119, 'aktif': 64, 'sd': 2467, 'november': 1938, 'pulang': 2253, 'hormat': 1077, 'sisa': 2593, 'kb': 1367, 'download': 726, 'mytelkomsel': 1836, 'apps': 154, 'kuotabeli': 1506, 'hubung': 1097, 'skb': 2601, 'ekstra': 778, 'pulsa': 2255, 'internet': 1176, 'bulan': 448, 'sjk': 2599, 'aug': 203, 'september': 2525, 'detail': 643, 'iring': 1198, 'tarif': 2749, 'panjang': 2049, 'hits': 1066, 'armada': 167, 'curi': 582, 'hati': 1031, 'tekan': 2777, 'okcall': 1989, 'informasi': 1147, 'eks': 775, 'loh': 1595, 'internetan': 1177, 'pakai': 2034, 'volume': 3032, 'ultima': 2970, 'mbak': 1692, 'harga': 1019, 'tariflokasi': 2751, 'tselmefl': 2914, 'coboy': 550, 'jr': 1284, 'hebat': 1039, 'baru': 269, 'noah': 1917,

In [80]:
# melihat jumlah feature
print(len(vec_TF_IDF.get_feature_names_out()))

3154


In [81]:
#melihat feature apa saja yang ada di dalam corpus
print(vec_TF_IDF.get_feature_names_out())

['aa' 'aamiiiin' 'aamiin' ... 'zarkasi' 'zona' 'zt']


In [74]:
x1 = vec_TF_IDF.transform(x).toarray() 
data_tabular_tf_idf = pd.DataFrame(x1,columns=vec_TF_IDF.get_feature_names_out())
data_tabular_tf_idf


,aa,aamiiiin,aamiin,ab,abadi,abai,abdul,acara,acaratks,account,...,yu,yudisium,yuk,yuks,yuni,yunit,zalora,zarkasi,zona,zt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
data_tabular_tf_idf.iloc[10:20,60:70]

,akbar,akreditasi,akses,aksi,aktif,aktifasi,aktivasi,aktivitas,akucintaislam,akumulasi
10,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
11,0.0,0.0,0.000000,0.0,0.154979,0.0,0.000000,0.0,0.0,0.0
12,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
13,0.0,0.0,0.262525,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
14,0.0,0.0,0.000000,0.0,0.244300,0.0,0.382803,0.0,0.0,0.0
15,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
16,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
17,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
18,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
19,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


## feature selection

In [83]:
x_train = np.array(data_tabular_tf_idf)
y_train = np.array(y)

In [84]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2_feature = SelectKBest(chi2, k=3000)
x_kbest_features = chi2_feature.fit_transform(x_train, y_train)

#untuk reduced feature
print('Orinal Feature Number', x_train.shape[1])
print('Reduce Feature Number', x_kbest_features.shape[1])

Orinal Feature Number 3154
Reduce Feature Number 3000


In [85]:
Data = pd.DataFrame(chi2_feature.scores_,columns=['Nilai'])
Data

,Nilai
0,1.307725
1,0.419755
2,1.561168
3,1.404246
4,0.765058
...,...
3149,1.764401
3150,1.195539
3151,0.479677
3152,3.030373


In [86]:
#menampilkan feature beserta nilainya

feature = vec_TF_IDF.get_feature_names_out()
feature

Data['Fitur'] = feature
Data

,Nilai,Fitur
0,1.307725,aa
1,0.419755,aamiiiin
2,1.561168,aamiin
3,1.404246,ab
4,0.765058,abadi
...,...,...
3149,1.764401,yunit
3150,1.195539,zalora
3151,0.479677,zarkasi
3152,3.030373,zona


In [88]:
#mengurutkan Nilai feature terbaik
Data.sort_values(by='Nilai', ascending=False)

,Nilai,Fitur
2036,49.752861,paket
992,47.188578,hadiah
1505,46.350120,kuota
2122,46.091967,pin
1286,38.260876,jt
...,...,...
822,0.034776,fb
1649,0.031319,maksimal
513,0.014601,cepat
3024,0.010008,via


In [89]:
mask = chi2_feature.get_support()
mask

array([ True,  True,  True, ...,  True,  True,  True])

In [90]:
# menampilkan fitur yang terpilih berdasarkan nilai mask atau nilai tertinggi yang sudah ditetapkan pada chi square

new_feature=[]
for bool, f in zip(mask, feature):
    if bool :
        new_feature.append(f)
    selected_feature=new_feature
selected_feature


['aa',
 'aamiiiin',
 'aamiin',
 'ab',
 'abadi',
 'abai',
 'abdul',
 'acaratks',
 'account',
 'ada',
 'adapromo',
 'adi',
 'adik',
 'adison',
 'admin',
 'administrasi',
 'adminlte',
 'ado',
 'adrian',
 'adu',
 'aduh',
 'advertising',
 'aesthetic',
 'af',
 'afr',
 'afrika',
 'agam',
 'agen',
 'agendain',
 'agenpulsa',
 'ags',
 'agst',
 'agsts',
 'agt',
 'agtskinfo',
 'agua',
 'agun',
 'agus',
 'agust',
 'agustuskunjungi',
 'ah',
 'ahaha',
 'ahub',
 'aidzin',
 'aigoo',
 'air',
 'aiu',
 'aja',
 'ajaa',
 'ajaaa',
 'ajabri',
 'ajak',
 'ajakin',
 'ajar',
 'ajeng',
 'akang',
 'akangteteh',
 'akbar',
 'akreditasi',
 'akses',
 'aksi',
 'aktif',
 'aktifasi',
 'aktivasi',
 'aktivitas',
 'akucintaislam',
 'akumulasi',
 'akun',
 'akurasi',
 'akurat',
 'ala',
 'alaikum',
 'alaikumsaya',
 'alaiqum',
 'alam',
 'alamat',
 'alami',
 'alamsyah',
 'alesannya',
 'algoritma',
 'alhamdulillah',
 'alhamdullilah',
 'alhuda',
 'ali',
 'aliando',
 'all',
 'allah',
 'allahaamiin',
 'alphard',
 'alquran',
 'alur',


In [91]:
#membuat vocabulary baru berdasarkan fitur-fitur yang terseleksi

new_selected_feature = {}
for (k,v) in vec_TF_IDF.vocabulary_.items():
    if k in selected_feature:
        new_selected_feature[k]=v

new_selected_feature

{'promo': 2222,
 'beli': 306,
 'paket': 2036,
 'flash': 842,
 'my': 1834,
 'telkomsel': 2783,
 'app': 149,
 'extra': 813,
 'kuota': 1505,
 'lte': 1607,
 'telpon': 2786,
 'buru': 462,
 'cek': 503,
 'tselmemytsel': 2916,
 'sk': 2600,
 'rupiah': 2412,
 'ribu': 2378,
 'spesial': 2658,
 'pilih': 2119,
 'aktif': 64,
 'sd': 2467,
 'november': 1938,
 'pulang': 2253,
 'hormat': 1077,
 'sisa': 2593,
 'kb': 1367,
 'download': 726,
 'mytelkomsel': 1836,
 'apps': 154,
 'kuotabeli': 1506,
 'hubung': 1097,
 'skb': 2601,
 'ekstra': 778,
 'pulsa': 2255,
 'internet': 1176,
 'bulan': 448,
 'sjk': 2599,
 'aug': 203,
 'september': 2525,
 'detail': 643,
 'iring': 1198,
 'tarif': 2749,
 'panjang': 2049,
 'hits': 1066,
 'armada': 167,
 'curi': 582,
 'hati': 1031,
 'tekan': 2777,
 'okcall': 1989,
 'informasi': 1147,
 'eks': 775,
 'loh': 1595,
 'internetan': 1177,
 'pakai': 2034,
 'volume': 3032,
 'ultima': 2970,
 'mbak': 1692,
 'harga': 1019,
 'tariflokasi': 2751,
 'tselmefl': 2914,
 'coboy': 550,
 'jr': 1284,

In [92]:
len(new_selected_feature)

3000

In [93]:
pickle.dump(new_selected_feature,open("new_selected_feature_tf-idf.sav","wb"))

In [94]:
new_selected_feature

{'promo': 2222,
 'beli': 306,
 'paket': 2036,
 'flash': 842,
 'my': 1834,
 'telkomsel': 2783,
 'app': 149,
 'extra': 813,
 'kuota': 1505,
 'lte': 1607,
 'telpon': 2786,
 'buru': 462,
 'cek': 503,
 'tselmemytsel': 2916,
 'sk': 2600,
 'rupiah': 2412,
 'ribu': 2378,
 'spesial': 2658,
 'pilih': 2119,
 'aktif': 64,
 'sd': 2467,
 'november': 1938,
 'pulang': 2253,
 'hormat': 1077,
 'sisa': 2593,
 'kb': 1367,
 'download': 726,
 'mytelkomsel': 1836,
 'apps': 154,
 'kuotabeli': 1506,
 'hubung': 1097,
 'skb': 2601,
 'ekstra': 778,
 'pulsa': 2255,
 'internet': 1176,
 'bulan': 448,
 'sjk': 2599,
 'aug': 203,
 'september': 2525,
 'detail': 643,
 'iring': 1198,
 'tarif': 2749,
 'panjang': 2049,
 'hits': 1066,
 'armada': 167,
 'curi': 582,
 'hati': 1031,
 'tekan': 2777,
 'okcall': 1989,
 'informasi': 1147,
 'eks': 775,
 'loh': 1595,
 'internetan': 1177,
 'pakai': 2034,
 'volume': 3032,
 'ultima': 2970,
 'mbak': 1692,
 'harga': 1019,
 'tariflokasi': 2751,
 'tselmefl': 2914,
 'coboy': 550,
 'jr': 1284,

In [95]:
#menampilkan fitur-fitur yang sudah diseleksi

data_selected_feature = pd.DataFrame(x_kbest_features, columns=selected_feature)
data_selected_feature

,aa,aamiiiin,aamiin,ab,abadi,abai,abdul,acaratks,account,ada,...,yu,yudisium,yuk,yuks,yuni,yunit,zalora,zarkasi,zona,zt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## modeling


In [96]:
selected_x = x_kbest_features
selected_x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [97]:
#import library
import random 
from sklearn.model_selection import train_test_split

#import algoritma naive bayes
from sklearn.naive_bayes import MultinomialNB

In [98]:
x = selected_x
y = data.label

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0) 

In [101]:
#menampilkan jumlah data training dan data testing
print('banyaknya X_train:', len(x_train))
print('banyaknya X_test:', len(x_test))
print('banyaknya y_train:', len(y_train))
print('banyaknya y_test:', len(y_test))




banyaknya X_train: 914
banyaknya X_test: 229
banyaknya y_train: 914
banyaknya y_test: 229


In [102]:
# procces training menggunakan naive bayes
text_algorithm = MultinomialNB()


In [103]:
model = text_algorithm.fit(x_train, y_train)

In [110]:
#membuat model prediksi

data_input = ("kantin fip nama pus bntar sholat")
data_input = text_preprocessing_process(data_input)

#load 
tfidf = TfidfVectorizer

loaded_vec = TfidfVectorizer(decode_error="replace", vocabulary=set(pickle.load(open("new_selected_feature_tf-idf.sav", "rb"))))

hasil = model.predict(loaded_vec.fit_transform([data_input]))

if(hasil==0):
    s = "SMS NORMAL"
elif(hasil==1):
    s = "SMS FRAUD"
else:
    s = "SMS PROMO"

print("Hasil Prediksi : \n", s)

Hasil Prediksi : 
 SMS NORMAL


## Evaluasi Model


In [111]:
## masukan  library
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

predicted = model.predict(x_test)

CM = confusion_matrix(y_test, predicted)

print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       126
           1       0.89      0.89      0.89        66
           2       0.85      0.89      0.87        37

    accuracy                           0.92       229
   macro avg       0.90      0.91      0.90       229
weighted avg       0.92      0.92      0.92       229



In [112]:
# menimpan model

pickle.dump(model,open("model_fraud.sav","wb"))


In [113]:
import random 
print(random.randint(1, 10))

1


In [115]:
pip install scikit-learn --upgrade
